### PyTorchとは

機械学習ライブラリ．   
Tensor(=torch.Tensor)というクラスを定義しており，多次元配列の保存と演算に利用している．これは，np.arrayに似ているが，これ以上に様々な演算が可能である．   
(参考：https://qiita.com/ya-ya-kaz716/items/bc13df58719f22332c28)

In [30]:
#ex.)
# Tensor
x_t = torch.tensor([[5,3],[10,6]])
# Numpy
x_n = np.array([[5,3],[10,6]])

print('Tensor:\n',x_t,'\n')
print('Numpy:\n',x_n)



Tensor:
 tensor([[ 5,  3],
        [10,  6]]) 

Numpy:
 [[ 5  3]
 [10  6]]


In [29]:
from gensim.models import KeyedVectors
import pandas as pd
import re
import torch
import pickle
import numpy as np

with open ("/Users/shirakawamomoko/Desktop/nlp100保存/chapter07/knock60.pkl","rb") as f:
    model = pickle.load(f)

In [79]:
def label_trans(cat):#i番目の事例ラベルyiの変換
    trans_dict={"b":0, "t":1, "e":2, "m":3}
    try:
        return trans_dict[cat]
    except:#対応するキーが見つからなかった場合
        return print(f"error: {cat}")


def Text2Vec(text):#特徴ベクトルの計算
    sp_t = text.split(" ")
    sigma_vec = 0#Σ
    length = 0#Ti
    for t in sp_t:
        try:
            temp = model.get_vector(t)
            sigma_vec += temp
            length += 1
        except:
            pass
    return sigma_vec/length#xi = (1/Ti)*∑t=1Tiemb(wi,t)


def torch_save(data,path):#行列・ベクトルの作成&ファイルに保存
    """
    data:処理対象のデータ名(拡張子を除く)
    path:dataの格納場所
    """
    df = pd.read_table(f"{path}{data}.txt")
    del_sign = re.compile('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`|＄＃＠£â€™]')#削除対象の記号の正規表現パターン
    del_func = lambda x: del_sign.sub("",x)#del_signをx(=入力された文字)から削除する無名関数
    df["TITLE"] = df["TITLE"].map(del_func)#title列にdel_funcを適用
    x_torch = torch.tensor(df["TITLE"].apply(Text2Vec))
    torch.save(x_torch,f"X_{data}.pt")#tensorを簡単に保存(.ptって何...??)
    df["CATEGORY"]=df["CATEGORY"].map(label_trans)
    y_torch = torch.tensor(df["CATEGORY"])
    torch.save(y_torch,f"Y_{data}.pt")
    print(f"Done!: {data}")

In [80]:
path = "/Users/shirakawamomoko/Desktop/nlp100保存/chapter06/"
torch_save("train",path)
torch_save("test",path)
torch_save("valid",path)

Done!: train
Done!: test
Done!: valid
